# Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json
import pickle

from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict

import warnings
warnings.filterwarnings('ignore')

# Load the Train Test Split

In [2]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12) 
Testing Samples : (83, 250, 12)
Training Labels : (332,) 
Testing Labels : (83,)


In [3]:
_ , num_timeframes, num_features = X_train.shape

# Reshape data for the model
-----------------
Samples are a 2D matrix which need to be reshaped to 1D to feed into the model

In [4]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 3000) 
Testing Samples : (83, 3000)
Training Labels : (332,) 
Testing Labels : (83,)


# Grid search over XGBoost model

In [5]:
parameters = {
    'max_depth': [8,9],
    'n_estimators': [12,14],
    'learning_rate': [0.2, 0.25],
    'gamma':[2.0, 2.5]
}
xgb = XGBClassifier(objective='multi:softmax', tree_method='auto', colsample_bytree=0.8, colsample_bylevel=0.8, colsample_bynode=0.8)
grid = GridSearchCV(xgb, parameters, cv=5, scoring='accuracy', verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=12 ......


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=12, total=   1.3s
[CV] gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=12 ......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV]  gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=12, total=   1.2s
[CV] gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=12 ......
[CV]  gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=12, total=   1.3s
[CV] gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=14 ......
[CV]  gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=14, total=   1.5s
[CV] gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=14 ......
[CV]  gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=14, total=   1.5s
[CV] gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=14 ......
[CV]  gamma=2.0, learning_rate=0.2, max_depth=8, n_estimators=14, total=   1.6s
[CV] gamma=2.0, learning_rate=0.2, max_depth=9, n_estimators=12 ......
[CV]  gamma=2.0, learning_rate=0.2, max_depth=9, n_estimators=12, total=   1.4s
[CV] gamma=2.0, learning_rate=0.2, max_depth=9, n_estimators=12 ......
[CV]  gamma=2.0, learning_rate=0.2, max_depth=9, n_estimators=12, total=   1.4s
[CV] gamma=2.0

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  1.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=0.8,
                                     colsample_bynode=0.8, colsample_bytree=0.8,
                                     gamma=None, gpu_id=None,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimato...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method='auto',
                                     validate_parameters=Fals

In [6]:
params = grid.best_params_
best_estimator = grid.best_estimator_
print(params)
print(best_estimator)

{'gamma': 2.5, 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 14}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=0.8,
              colsample_bynode=0.8, colsample_bytree=0.8, gamma=2.5, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.25, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=14, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='auto', validate_parameters=False, verbosity=None)


In [7]:
clf = best_estimator
clf.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=0.8,
              colsample_bynode=0.8, colsample_bytree=0.8, gamma=2.5, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.25, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=14, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='auto', validate_parameters=False, verbosity=None)

In [8]:
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.65      0.79      0.71        14
           1       0.90      0.64      0.75        14
           2       0.79      0.85      0.81        13
           3       0.94      1.00      0.97        16
           4       0.73      0.79      0.76        14
           5       0.80      0.67      0.73        12

    accuracy                           0.80        83
   macro avg       0.80      0.79      0.79        83
weighted avg       0.80      0.80      0.79        83

[[11  1  1  0  1  0]
 [ 3  9  0  1  1  0]
 [ 1  0 11  0  1  0]
 [ 0  0  0 16  0  0]
 [ 1  0  0  0 11  2]
 [ 1  0  2  0  1  8]]


In [9]:
pickle.dump(clf, open(os.path.join("..", "IPD", "XGBoost_model_half.pkl"), 'wb'))
print("Model Saved")

Model Saved


# Train on entire data

In [13]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))

X_train = np.concatenate((X_train,X_test), axis=0)
Y_train = np.concatenate((Y_train,Y_test), axis=0)

_ , num_timeframes, num_features = X_train.shape
X_train = X_train.reshape((-1,num_timeframes*num_features))
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(X_train)

from sklearn.utils import shuffle
X_train, X_sparse, Y_train = shuffle(X_train, X_sparse, Y_train)
X_train = X_train.reshape((-1,num_timeframes,num_features))

In [14]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (415, 3000) 
Testing Samples : (83, 3000)
Training Labels : (415,) 
Testing Labels : (83,)


In [15]:
parameters = {
    'max_depth': [6,8,10],
    'n_estimators': [15, 18, 20, 23, 25],
    'learning_rate': [0.15, 0.2, 0.25],
    'gamma':[1.5, 0.2, 0.25]
}
xgb = XGBClassifier(objective='multi:softmax', tree_method='auto', colsample_bytree=0.8, colsample_bylevel=0.8, colsample_bynode=0.8)
grid = GridSearchCV(xgb, parameters, cv=5, scoring='accuracy', verbose=2)
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 135 candidates, totalling 675 fits
[CV] gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15 .....


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15, total=   2.1s
[CV] gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15 .....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV]  gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15, total=   2.1s
[CV] gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15 .....
[CV]  gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15, total=   2.2s
[CV] gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15 .....
[CV]  gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15, total=   2.2s
[CV] gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15 .....
[CV]  gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=15, total=   2.3s
[CV] gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=18 .....
[CV]  gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=18, total=   2.6s
[CV] gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=18 .....
[CV]  gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=18, total=   2.6s
[CV] gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=18 .....
[CV]  gamma=1.5, learning_rate=0.15, max_depth=8, n_estimators=18, total=   2.6s
[CV] ga

...

[CV]  gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=23, total=   3.3s
[CV] gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=23 .....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=23, total=   3.2s
[CV] gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=23 .....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=23, total=   3.2s
[CV] gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=23 .....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=23, total=   3.2s
[CV] gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=25 .....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=25, total=   3.3s
[CV] gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=25 .....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=25, total=   3.4s
[CV] gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=25 .....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=8, n_estimators=25, total=   3.3s
[CV] ga

[CV]  gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25, total=   3.4s
[CV] gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25 ....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25, total=   3.5s
[CV] gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25 ....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25, total=   3.4s
[CV] gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25 ....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25, total=   3.5s
[CV] gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25 ....
[CV]  gamma=0.25, learning_rate=0.2, max_depth=12, n_estimators=25, total=   3.4s
[CV] gamma=0.25, learning_rate=0.25, max_depth=8, n_estimators=15 ....
[CV]  gamma=0.25, learning_rate=0.25, max_depth=8, n_estimators=15, total=   2.1s
[CV] gamma=0.25, learning_rate=0.25, max_depth=8, n_estimators=15 ....
[CV]  gamma=0.25, learning_rate=0.25, max_depth=8, n_estimators=15, total=   2.1s


[CV]  gamma=0.25, learning_rate=0.25, max_depth=10, n_estimators=25, total=   3.2s
[CV] gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15 ...
[CV]  gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15, total=   2.2s
[CV] gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15 ...
[CV]  gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15, total=   2.1s
[CV] gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15 ...
[CV]  gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15, total=   2.1s
[CV] gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15 ...
[CV]  gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15, total=   2.2s
[CV] gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15 ...
[CV]  gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=15, total=   2.1s
[CV] gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=18 ...
[CV]  gamma=0.25, learning_rate=0.25, max_depth=12, n_estimators=18, total= 

[Parallel(n_jobs=1)]: Done 675 out of 675 | elapsed: 31.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=0.8,
                                     colsample_bynode=0.8, colsample_bytree=0.8,
                                     gamma=None, gpu_id=None,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimato...
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method='auto',
                                     validate_parameters=False,
                                     verbosity=None),
             ii

In [16]:
clf = grid.best_estimator_
clf.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=0.8,
              colsample_bynode=0.8, colsample_bytree=0.8, gamma=0.25, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.25, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='auto', validate_parameters=False, verbosity=None)

# Saving model

In [17]:
pickle.dump(clf, open(os.path.join("..", "models", "XGBoost_model.pkl"), 'wb'))
print("Model Saved")

Model Saved
